Czym może być perturbacja:
 - najprostsza - wymiana x wierzchołków na inne nieużywane (losowo)

 - wybierz losowo x wierzchołków i zastąp je najbliższym sąsiadem poprzedniego

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from typing import List, Dict, Tuple
import copy
import time
import numpy as np
from joblib import Parallel, delayed
from itertools import combinations, product
from queue import PriorityQueue, Empty

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from typing import List, Dict, Tuple
import copy
import numpy as np

MAX_DIST = 2147483647

def create_distance_matrix(df):
    x = df['x'].values
    y = df['y'].values
    x1 = x.reshape((df.shape[0], 1))
    x2 = x.reshape((1, df.shape[0]))
    y1 = y.reshape((df.shape[0], 1))
    y2 = y.reshape((1, df.shape[0]))
    matrix = np.round(np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)).astype(int)
    np.fill_diagonal(matrix, MAX_DIST)
    return matrix

def evaluate(distance_matrix: np.ndarray, path: np.array, costs: np.array) -> int:
    copy_path = np.array(path)
    copy_path = np.append(copy_path, copy_path[0])
    distances = distance_matrix[copy_path[:-1], copy_path[1:]]
    path_costs = costs[copy_path[:-1]]
    total_length = np.sum(distances + path_costs)
    return total_length

def random_sequence(distance_matrix: np.ndarray, start_node: int = 0) -> List[int]:
    n = distance_matrix.shape[0]
    num_selected = n // 2 if n % 2 == 0 else (n // 2) + 1
    path = random.sample(list(range(distance_matrix.shape[0])), num_selected)
    return np.array(path)

def get_plot_values(nodes : Dict[int, Tuple[int, int, int]], solution: List[int], costs: List[int]) -> Tuple[List[int], List[int], List[int], List[int], List[int]]:
    x_coords = [nodes[node][0] for node in list(nodes.keys())]
    y_coords = [nodes[node][1] for node in list(nodes.keys())]
    solution = solution + [solution[0]]
    path_x_coords = [nodes[node][0] for node in solution]
    path_y_coords = [nodes[node][1] for node in solution]
    new_costs = [(cost/max(costs))*100 for cost in costs]
    min_cost, max_cost = min(costs), max(costs)
    power = 2
    normalized_costs = [((cost - min_cost) / (max_cost - min_cost)) ** power for cost in costs]
    colors = plt.cm.RdBu(normalized_costs)
    return x_coords, y_coords, new_costs, path_x_coords, path_y_coords, colors

def plot_path(path, nodes, costs):
    x_coords, y_coords, new_costs, path_x_coords, path_y_coords, colors = get_plot_values(nodes, path, costs)
    plt.scatter(x_coords, y_coords, color=colors, marker='o', s=new_costs, label='Cities')
    plt.plot(path_x_coords, path_y_coords, linestyle='-', marker='o', markersize=0, color='blue', label='Path', alpha = 0.7)
    plt.show()

def read_data(filename='TSPA.csv', PATH='./'):
    df = pd.read_csv(PATH + filename, names=["x", "y", "costs"], sep=';', header=None)
    nodes = {}
    for idx, row in enumerate(df.values):
        x, y, cost = map(int, row)
        nodes[idx] = (x, y, cost)
    D = create_distance_matrix(df)
    return nodes, df['costs'].values, D

In [ ]:
class Delta:
  def __init__(self, delta_value: int, delta_type: str, values: tuple):
      self.values = values
      self.delta_type = delta_type
      self.delta_value = delta_value

  def create_path(self, path):
      if self.delta_type == 'inter':
          index_of_old_node, new_value = self.values
          modified_path = copy.deepcopy(path)
          modified_path[index_of_old_node] = new_value
          return modified_path
      elif self.delta_type == 'edges_intra':
          n1, n2 = self.values
          if n1 < n2:
              modified_path = np.concatenate((path[:n1], [path[n1]], [path[n2]], path[n1+1:n2][::-1],  path[n2+1:]))
          else:
              modified_path = np.concatenate((path[n2+1:n1], [path[n1]], [path[n2]], path[:n2][::-1], path[n1+1:][::-1]))
          return modified_path
      else:
          raise ValueError("Invalid delta_type!")

In [ ]:
def inter(path, distance_matrix, costs):
    og_nodes = set(path)
    nodes = set(list(range(distance_matrix.shape[0])))
    changes = product(og_nodes, nodes - og_nodes)
    changes = list(changes)

    random.shuffle(changes)

    node_indices = {node: index for index, node in enumerate(path)}

    neighbourhood_deltas = []

    for change in changes:
        old_node, new_node = change
        delta_value = 0
        index_of_old_node = node_indices[old_node]

        prev = path[(index_of_old_node - 1) % len(path)]
        next = path[(index_of_old_node + 1) % len(path)]

        old_dist = distance_matrix[prev, old_node] \
                  + distance_matrix[old_node, next]
        new_dist = distance_matrix[prev, new_node] \
                  + distance_matrix[new_node, next]
        delta_value += new_dist - old_dist  # lower is better

        delta_value += costs[new_node] - costs[old_node]  # lower is better

        delta = Delta(delta_value, 'inter', (index_of_old_node, new_node))

        if delta_value < 0:
            return [delta]

        neighbourhood_deltas.append(delta)
    return []

def edges_intra(path, distance_matrix):
    pairs1 = np.column_stack((path[:-1], path[1:]))

    cyclic_pair = np.array([path[-1], path[0]])

    all_edges = np.vstack((pairs1, cyclic_pair))

    edge_swaps = combinations(all_edges, 2)

    edge_swaps = list(edge_swaps)

    random.shuffle(edge_swaps)

    neighbourhood_deltas = []

    node_indices = {node: index for index, node in enumerate(path)}
    # remove non valid edge swaps
    for swap in edge_swaps:
        edge1, edge2 = swap
        if edge1[1] == edge2[0] or edge1[0] == edge2[1]:
            continue
        delta_value = (
            distance_matrix[edge1[0], edge2[0]]
            + distance_matrix[edge1[1], edge2[1]]
        ) - (
            distance_matrix[edge1[0], edge1[1]]
            + distance_matrix[edge2[1], edge2[0]]
        )
        index1 = node_indices[edge1[0]]
        index2 = node_indices[edge2[0]]

        delta = Delta(delta_value, 'edges_intra', (index1, index2))

        if delta_value < 0:
            return [delta]

        neighbourhood_deltas.append(delta)
    return []

In [ ]:
def local_search(initial_solution, costs, distance_matrix):
    if initial_solution is None:
        initial_solution = random_sequence(distance_matrix)

    initial_evaluation = evaluate(distance_matrix, initial_solution, costs)
    current_path = initial_solution
    i = 0
    while True:
        i += 1
        print(current_path)
        neighbourhood_inter_deltas = inter(current_path, distance_matrix, costs)
        neighbourhood_intra_deltas = edges_intra(current_path, distance_matrix)
        neighbourhood_deltas = np.array(neighbourhood_inter_deltas + neighbourhood_intra_deltas)
        if neighbourhood_deltas.size > 0:
            best_delta = np.random.choice(neighbourhood_deltas)
            print(best_delta.delta_type)
            print(best_delta.delta_value)
            current_path = best_delta.create_path(current_path)
        else:
            break

    final_evaluation = evaluate(distance_matrix, current_path, costs)
    return initial_evaluation, current_path, final_evaluation

In [ ]:
import copy
def perturbation_replacenn(D, costs, cycle, n=5):
    random_indices = random.sample(list(range(len(cycle))), n)
    copy_cycle = copy.deepcopy(cycle)
    for i in range(n):
        next_index = (random_indices[i] + 1) % len(cycle)
        distances = copy.deepcopy(D[copy_cycle[random_indices[i]], :]) + costs
        distances[copy_cycle] = MAX_DIST
        copy_cycle[next_index] = np.argmin(distances)
    return copy_cycle

def perturbation_swap5(D, cycle, n=5):
    og_nodes = set(cycle)
    nodes = set(list(range(D.shape[0])))
    unused = nodes - og_nodes
    copy_cycle = copy.deepcopy(cycle)
    for i in range(n):
        random_unused = random.choice(list(unused))
        unused = unused - set([random_unused])
        random_index = random.choice(list(range(len(cycle))))
        copy_cycle[random_index] = random_unused
    return copy_cycle

def ILS(initial, costs, D, time_limit):
    time_start = time.time()
    time_running = 0
    init, best_path, best_cost = local_search(initial, costs, D)
    while True:
        perturbed_path = perturbation_swap5(D, best_path)
        init, ls_path, ls_score = local_search(perturbed_path, costs, D)
        if ls_score < best_cost:
            best_cost = ls_score
            best_path = copy.deepcopy(ls_path)
        time_running = time.time() - time_start
        print(time_running)
        if time_running > time_limit:
            break
    return best_path, best_cost

def ILS2(initial, costs, D, time_limit):
    time_start = time.time()
    time_running = 0
    init, best_path, best_cost = local_search(initial, costs, D)
    while True:
        perturbed_path = perturbation_replacenn(D, costs, best_path)
        init, ls_path, ls_score = local_search(perturbed_path, costs, D)
        if ls_score < best_cost:
            best_cost = ls_score
            best_path = copy.deepcopy(ls_path)
        time_running = time.time() - time_start
        print(time_running)
        if time_running > time_limit:
            break
    return best_path, best_cost

In [ ]:
nodes, costs, D = read_data(filename="TSPA.csv")

In [ ]:
D[:10, :10]

array([[2147483647,       1549,        636,        480,        615,
              1950,       1950,       1614,       2433,       1303],
       [      1549, 2147483647,        936,       2027,        947,
              1331,       1924,       2824,       1288,        738],
       [       636,        936, 2147483647,       1115,         35,
              1443,       1656,       1966,       1831,        717],
       [       480,       2027,       1115, 2147483647,       1095,
              2357,       2239,       1441,       2887,       1759],
       [       615,        947,         35,       1095, 2147483647,
              1478,       1689,       1973,       1861,        750],
       [      1950,       1331,       1443,       2357,       1478,
        2147483647,        768,       2389,        743,        761],
       [      1950,       1924,       1656,       2239,       1689,
               768, 2147483647,       1829,       1485,       1205],
       [      1614,       2824,       196

In [ ]:
costs[:10]

array([  84,  483, 1462, 1986,  145, 1117,  151, 1072,  273, 1589])

In [ ]:
random_sol = random_sequence(D)

In [ ]:
random_sol

array([192,  48,  80,  50,  81, 196,  11, 141,  24,  98,  57, 131, 174,
       172, 139,  84,  13, 114, 119,  31,  28, 176,   1, 181, 103,  83,
       137, 132, 146,   5, 155,   7, 105, 126,  47, 124, 112, 148,  29,
        23, 163,  25,  15,  87,  40,  26, 194,  36,  54, 108,  14, 121,
       122,   2, 188, 111,  41, 177,  17,  74,  18, 162, 189, 106, 179,
       170, 123, 185, 167, 173,  43, 191,  46,  93, 145,  72,  10,  39,
       175, 197,  22,   4,  45, 154, 178,  99, 150, 128,  52,  34, 136,
         8, 130,  42,   9,  85, 135, 129,  30,  64])

In [ ]:
ILS(random_sol, costs, D, 100)

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
-168
[153 186  45  51 135 101 167 175   2   4 114  91 121  50  43 150  77 192
 199 137 177   1 130 119 109 189  75 174  41 152  11  48 106  26   8 123
  80 124 169  95 112  73  31  14 111  94  12  72 190  98  66 156   6  24
 141 144  87 102 154  81 131 180  32  62 155 163  74 113  61  71  20  64
 185  96  27 147  59 143 159 178 164  19 149   0 128 132  36  55 195  22
  53 117 108 171  76 161  21 194  79  88]
edges_intra
-146
[153 186  45  51 135 101 167 175   2   4 114  91 121  50  43 150  77 192
 199 137 177   1 130 119 109 189  75 174  41 152  11  48 106  26   8 123
  80 124 169  95 112  73  31  14 111  94  12  72 190  98  66 156   6  24
 141 144  87 102 154  81 131 180  32  62 155 163  74 113  61  71  20  64
 185  96  27 147  59 143 159 164 178  19 149   0 128 132  36  55 195  22
  53 117 108 171  76 161  21 194  79  88]
edges_intra
-535
[153 186  45  51 135 101 167 175   2   4 114  91 121  50  43 150  77 192
 199 137 

(array([156,   6,  24, 141,  87, 144, 154,  81, 180,  32,  62,  53,  22,
        195, 155, 163,  74, 113,  61,  71,  20,  64, 185,  96,  27, 147,
         59, 143, 159,  34, 164, 178,  19,  69,   0, 149,  50,  43,  77,
          4, 114, 121,  91,  76, 145, 128, 132,  36,  55, 117,  15, 108,
        171,  21, 194,  79, 186, 127,  88, 153, 167, 175, 192, 150, 199,
        137,  41, 177,   1,  75, 189, 109, 119, 130, 152,  11, 160, 198,
        106,  48,  26,   8, 124,  80, 169,  95, 135,  51,   5, 112,  73,
         31,  14, 111,  94,  12,  72, 190,  98,  66]),
 74703)

In [ ]:
ILS2(random_sol, costs, D, 100)

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
[ 43 121  91 149   0  19 178 164 128  37 159 143  59  96 147  27  25 185
  64  20  71 183  61 113  74 163 155  93  62  32 180  81 154 102 144 141
  87  79 194  21 171 108 117  53  22 195  55  36 132  49  76 161 153  88
 186  45  24   6 156  66  98 190  72  94  73  31 111  14  80  95 169   8
  26  92  48 106 160  11 152   1 177  41 137 199 174  75 189 109 130 119
  51 112  99 135 167 175   4 114 192  77]
inter
-7
[ 43 121  91 149   0  19 178 164 128  37 159 143  59  96 147  27  25 185
  64  20  71 183  61 113  74 163 155  93  62  32 180  81 154 102 144 141
  87  79 194  21 171 108 117  53  22 195  55  36 132 118  76 161 153  88
 186  45  24   6 156  66  98 190  72  94  73  31 111  14  80  95 169   8
  26  92  48 106 160  11 152   1 177  41 137 199 174  75 189 109 130 119
  51 112  99 135 167 175   4 114 192  77]
edges_intra
-17
[ 43 121  91 149   0  19 178 164 128  37 159 143  59  96 147  27  25 185
  64  20  71 183  61 11

(array([ 43, 192, 175, 167, 101,  99, 135, 134, 119, 130, 109, 189,  75,
        174, 199,  41, 177,   1, 152,  11, 160, 106,  48,  92,  26,   8,
        169,  95,  80,  14, 111,  31,  73,  94,  72, 190,  98,  66, 156,
          6,  24, 186, 127,  88, 153, 161,  76, 145, 128, 132,  36,  55,
        195,  22,  53, 117,  15, 108, 171,  21, 194,  79,  87, 141, 144,
        154,  81, 180,  32,  62, 155, 163,  74, 113, 181,  61, 183,  71,
         20,  64, 185, 116,  27, 147,  96,  59, 143, 159, 164, 178,  19,
         69,   0, 149,  50, 121,  91, 114,   4,  77]),
 73930)